In [0]:
import requests
import pandas as pd

from bs4 import BeautifulSoup

In [0]:
base_url = 'https://www.grammy.com'
awards_url = 'https://www.grammy.com/grammys/awards'

In [0]:
req = requests.get(awards_url)

In [0]:
req.status_code

200

In [0]:
soup = BeautifulSoup(req.content, 'html.parser')

In [0]:
awards = soup.find(class_='views-infinite-scroll-content-wrapper')

In [0]:
rows = awards.find_all(class_='views-row')

In [0]:
img = rows[0].a.img['src']

In [0]:
last_edition_url = f"{base_url}{rows[0].a['href']}"
print(last_edition_url)

https://www.grammy.com/grammys/awards/62nd-annual-grammy-awards-2019


In [0]:
last_year = last_edition_url.split('-')[4]
print(last_year)

2019


In [0]:
req = requests.get(last_edition_url)
req.status_code

200

In [0]:
soup = BeautifulSoup(req.content, 'html.parser')
all_editions_url = soup.find(class_='view-show-year').find_all(class_='views-row')
len(all_editions_url)

62

In [0]:
df = pd.DataFrame(columns=['year', 'title', 'published_at', 'updated_at', \
                           'category', 'nominee', 'artist', 'workers', \
                           'img', 'winner'])

In [0]:
def insert_row(df, row):
    insert_loc = df.index.max()

    if pd.isna(insert_loc):
        df.loc[0] = row
    else:
        df.loc[insert_loc + 1] = row

In [0]:
for edition_url in all_editions_url[1:]:
  url = f"{base_url}{edition_url.a['href']}"

  req = requests.get(url)
  if req.status_code == 200:
    soup = BeautifulSoup(req.content, 'html.parser')

    title = soup.find(class_='sub-title').h2.string
    published_at = soup.find("meta",  property="article:published_time")['content']
    updated_at = soup.find("meta",  property="article:modified_time")['content']
    year = soup.find(id='page-title').getText().split(" ")[0]

    articles = soup.find_all(class_='view-grouping')
    for article in articles:
      category = article.find(class_='view-grouping-header').string

      try:
        img = article.find(class_='circular-image').img['src']
      except:
        img = ''

      winner = True
      nominee = article.find(class_='views-field-title').string

      try:
        artist = article.find(class_='views-field-field-description').p.getText()
      except:
        artist = ''

      try:
        workers = article.find(class_='views-field-body-1').p.getText()
      except:
        workers = ''

      insert_row(df, [year, title, published_at, updated_at, category, nominee, \
                  artist, workers, img, winner])

In [0]:
df.loc[df.artist == '']

In [0]:
new_df = df.drop(['year', 'published_at', 'updated_at', 'winner'], axis=1)

In [0]:
human_awards = ['Best New Artist', '']

In [0]:
new_df.loc[new_df.category == 'Best New Artist']

In [0]:
new_df['category'].unique()

In [0]:
new_df.to_csv('grammy.csv', index=False, sep=',')

In [0]:
new_df.winner = True

In [27]:
new_df.loc[new_df.winner == True]

,title,category,nominee,artist,workers,img,winner
0,61st Annual GRAMMY Awards (2018),Record Of The Year,This Is America,Childish Gambino,"Donald Glover & Ludwig Göransson, producers; D...",https://www.grammy.com/sites/com/files/styles/...,True
1,61st Annual GRAMMY Awards (2018),Album Of The Year,Golden Hour,Kacey Musgraves,"Ian Fitchuk, Kacey Musgraves & Daniel Tashian,...",https://www.grammy.com/sites/com/files/styles/...,True
2,61st Annual GRAMMY Awards (2018),Song Of The Year,This Is America,,"Donald Glover, Ludwig Göransson & Jeffery Lama...",https://www.grammy.com/sites/com/files/styles/...,True
3,61st Annual GRAMMY Awards (2018),Best New Artist,Dua Lipa,,,,True
4,61st Annual GRAMMY Awards (2018),Best Pop Solo Performance,Joanne (Where Do You Think You're Goin'?),Lady Gaga,,https://www.grammy.com/sites/com/files/styles/...,True
...,...,...,...,...,...,...,...
4341,1st Annual GRAMMY Awards (1958),Best Classical Performance - Instrumentalist (...,Tchaikovsky: Piano Concerto No. 1 In B Flat Mi...,,"Van Cliburn, artist (Symphony Of The Air Orche...",,True
4342,1st Annual GRAMMY Awards (1958),Best Classical Performance - Instrumentalist (...,Segovia Golden Jubilee,,"Andres Segovia, artist",https://www.grammy.com/sites/com/files/styles/...,True
4343,1st Annual GRAMMY Awards (1958),Best Classical Performance - Chamber Music (In...,Beethoven: Quartet 130,,"Hollywood String Quartet (Alvin Dinkin, Paul S...",,True
4344,1st Annual GRAMMY Awards (1958),Best Classical Performance - Vocal Soloist (Wi...,Operatic Recital,,,,True
